Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [20]:
batch_size = 100
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # first convolution
    conv1 = conv2d(data, layer1_weights, layer1_biases)
    conv1 = max_pool(conv1, 2)
    conv1 = tf.nn.dropout(conv1, 0.75)
    # sencond convolution
    conv2 = conv2d(conv1, layer2_weights, layer2_biases)
    conv2 = max_pool(conv2, 2)
    conv2 = tf.nn.dropout(conv2, 0.75)
    # reshaping
    shape = conv2.get_shape().as_list()
    dense1 = tf.reshape(conv2, [shape[0], shape[1] * shape[2] * shape[3]])
    dense1 = tf.nn.relu(tf.matmul(dense1, layer3_weights) + layer3_biases)
    
    # output
    return tf.matmul(dense1, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  regularizer = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                 tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                 tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                 tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases))
  
  loss += 0.1 * regularizer
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [21]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
        print "Iter {:8}".format(step*batch_size) + ", Minibatch Loss:" + \
              "{:.6f}".format(l) + ", Training Accuracy:" + "{:3.2f}".format(accuracy(predictions, batch_labels)) + \
              ", Validation Accuracy:" + "{:3.2f}".format(accuracy(valid_prediction.eval(), valid_labels))
      #print "Minibatch loss at step", step, ":", l
      #print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      #print "Validation accuracy: %.1f%%" % accuracy(
        #valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Iter        0, Minibatch Loss:30.664934, Training Accuracy:9.00, Validation Accuracy:9.42
Iter    20000, Minibatch Loss:19.461525, Training Accuracy:49.00, Validation Accuracy:48.67
Iter    40000, Minibatch Loss:13.396885, Training Accuracy:62.00, Validation Accuracy:66.60
Iter    60000, Minibatch Loss:9.335921, Training Accuracy:70.00, Validation Accuracy:72.12
Iter    80000, Minibatch Loss:6.695230, Training Accuracy:76.00, Validation Accuracy:74.32
Iter   100000, Minibatch Loss:5.097852, Training Accuracy:69.00, Validation Accuracy:76.21
Iter   120000, Minibatch Loss:3.521293, Training Accuracy:84.00, Validation Accuracy:76.52
Iter   140000, Minibatch Loss:2.978170, Training Accuracy:74.00, Validation Accuracy:77.68
Iter   160000, Minibatch Loss:2.220038, Training Accuracy:80.00, Validation Accuracy:77.21
Iter   180000, Minibatch Loss:1.990779, Training Accuracy:78.00, Validation Accuracy:78.26
Iter   200000, Minibatch Loss:1.995546, Training Accuracy:71.00, Validation A

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---